# 通信シミュレーション（受信側）

情報理論における符号化を学ぶため，雑音のある通信路を経由してメッセージを受信してみましょう．

## 準備

ストレージをマウントします．通信路として使用します．

In [29]:
try:
    from google.colab import drive
    IN_COLAB = True
    print("Running on Google Colab")
    drive.mount('/content/drive')
    saved_dir = '/content/drive/MyDrive/info-theory'
except ImportError:
    IN_COLAB = False
    print("Not running on Google Colab")
    saved_dir = '//nas/info-theory'

os.makedirs(saved_dir, exist_ok=True)
print(f"Saved directory is set to: {saved_dir}")

import os
print("current directory", os.getcwd())
import sys
sys.path.append("/content/drive/MyDrive")
# GitHubリポジトリのURL
# !wget -q https://raw.githubusercontent.com/shizoda/education/main/info_theory/communication_lib.py

# 必要なPythonファイルをインポートする
import communication_lib

Running on Google Colab
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved directory is set to: /content/drive/MyDrive/info-theory
current directory /content


## メッセージ

送り主，宛先，メッセージ本文を指定します．

なお，サンプルの英文は "e"，和文は「静」や「岡」が多用された文になっています．

In [30]:
message_from = "alice"
message_to   = "bob"

In [ ]:
def receive_message(saved_dir, orig, other):
    
    # 相手から送信されたビットを受信
    data_filename = f"{saved_dir}/{orig}-{other}.txt"
    codebook_filename = f"{saved_dir}/{orig}-{other}-codebook.json"
    padding_filename = f"{saved_dir}/{orig}-{other}-padding.txt"
    codebook = load_codebook(codebook_filename)
    padding_length = load_padding_length(padding_filename)
    received_data = load_from_file(data_filename, orig, other)
    if received_data is None:
        return None
    
    # 正しく並べ替えてください
    #### ここから
    
    print("\n 〇〇〇△△器：□□□□符号") 
    hamming_decoded_data = hamming_decode(received_data)
    
    print("\n 雑音のある通信路から受信")
    print(f"Received      : {format_binary_string(received_data)}")
    
    print("\n 〇〇〇△△器：□□□□符号")
    decoded_message = huffman_decoding(hamming_decoded_data, codebook, padding_length)
    
    #### ここまで
    
    return decoded_message

In [ ]:
other = message_from
orig = message_to

# 相手から送信されたビットを受信
data_filename = f"{saved_dir}/{orig}-{other}.txt"
codebook_filename = f"{saved_dir}/{orig}-{other}-codebook.json"
padding_filename = f"{saved_dir}/{orig}-{other}-padding.txt"
codebook = load_codebook(codebook_filename)
padding_length = load_padding_length(padding_filename)
received_data = load_from_file(data_filename, orig, other)
if received_data is None:
    return None

In [ ]:
# 正しく並べ替えてください
#### ここから

print("\n 〇〇〇△△器：□□□□符号") 
hamming_decoded_data = hamming_decode(received_data)

In [ ]:
print("\n 雑音のある通信路から受信")
print(f"Received      : {format_binary_string(received_data)}")

In [ ]:
print("\n 〇〇〇△△器：□□□□符号")
decoded_message = huffman_decoding(hamming_decoded_data, codebook, padding_length)

#### ここまで

In [31]:
received_message = decoded_message
print(f"Final Decoded Message: {received_message}")


 雑音のある通信路から受信
Received      : 0001100 0001100 0110000 0111111 0110110 1000010 0111101 0...

 〇〇〇△△器：□□□□符号
220 corrections in total.
  (Block-0, Bit-0): 0001100 --> 1001100
  (Block-1, Bit-0): 0001100 --> 1001100
  (Block-2, Bit-0): 0110000 --> 1110000
  (Block-3, Bit-0): 0111111 --> 1111111
  (Block-4, Bit-1): 0110110 --> 0010110
  (Block-5, Bit-6): 1000010 --> 1000011
  (Block-6, Bit-6): 0111101 --> 0111100
  (Block-8, Bit-3): 1100001 --> 1101001
  (Block-9, Bit-4): 0011101 --> 0011001
  (Block-10, Bit-5): 0110001 --> 0110011
      etc.
Received       : 0001100 0001100 0110000 0111111 0110110 1000010 0111101 0...
Hamming Decoded: 1001100 1001100 1110000 1111111 0010110 1000011 0111100 0...
Hamming Output : 0100 0100 1000 1111 1110 0011 1100 1110 0001 1001 1011 1100 01.. .

 〇〇〇△△器：□□□□符号

Codebook:  e:00  :011 i:0101 l:1010 s:1011 n:1110 E:01000 a:01001 c:10000 v:10010 t:10011 y:11000 g:11010 r:11111 p:100010 ,:100011 .:110010 x:110111 o:111100 m:1100110 d:1101100 u:1101101 h:111101